# Lab 02 - Basics of Image Processing

**ESTIN S4 (DS-AI) 2025-2026**

This notebook implements a comprehensive set of image processing functions including:

**Exercise 1:**
- OpenImage, Divide, HSV, CountPix, FactPix
- Func_a (logarithm, exponential, square, square root transformations)
- Func_m (mean and standard deviation)
- Normalize, Inverse, CalcHist, Threshold
- Complete processing pipelines (Func_j, Func_t, Func_f)

**Exercise 2:**
- Matrix definition and visualization
- Random matrix generation and analysis

## Import Libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

## Exercise 1: Basic Image Processing Functions

### 1.1 OpenImage Function

Takes a single argument (image path) and returns three variables:
- I: the image matrix
- L: the number of rows
- C: the number of columns

In [ ]:
def OpenImage(path):
    """
    Takes a single argument (image path) and returns three variables:
    - I: the equivalent matrix (image)
    - L: the number of rows in the matrix
    - C: the number of columns
    """
    I = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    if I is None:
        raise ValueError(f"Unable to load image from path: {path}")
    L, C = I.shape
    return I, L, C

# Test the function
# I, L, C = OpenImage('your_image.jpg')
# print(f"Image loaded: {L}x{C} pixels")

### 1.2 Divide, HSV, CountPix, and FactPix Functions

In [ ]:
def Divide(image):
    """Returns the three separate channels (B, G, R) of an image."""
    if len(image.shape) == 2:
        return image, image, image
    B, G, R = cv2.split(image)
    return B, G, R


def HSV(image):
    """Returns the HSV format of an image."""
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    return hsv_image


def CountPix(image):
    """Returns N (the number of pixels in the image)."""
    if len(image.shape) == 2:
        L, C = image.shape
    else:
        L, C, _ = image.shape
    N = L * C
    return N


def FactPix(image, alpha=1.0, beta=0):
    """
    Modifies the intensities using: I_fact = alpha * I + beta
    - alpha: contrast control (multiplication factor)
    - beta: brightness control (addition factor)
    """
    I_fact = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
    return I_fact

### 1.3 Func_a and Func_m Functions

**Func_a**: Returns logarithm, exponential, square, and square root transformations  
**Func_m**: Returns mean μ and standard deviation σ

In [ ]:
def Func_a(image):
    """Returns log, exponential, square, and square root of the input image."""
    image_float = image.astype(np.float64)
    
    # Logarithm (add 1 to avoid log(0))
    log_image = np.log1p(image_float)
    log_image = np.uint8(cv2.normalize(log_image, None, 0, 255, cv2.NORM_MINMAX))
    
    # Exponential
    exp_image = np.exp(image_float / 255.0)
    exp_image = np.uint8(cv2.normalize(exp_image, None, 0, 255, cv2.NORM_MINMAX))
    
    # Square
    square_image = np.square(image_float)
    square_image = np.uint8(cv2.normalize(square_image, None, 0, 255, cv2.NORM_MINMAX))
    
    # Square root
    sqrt_image = np.sqrt(image_float)
    sqrt_image = np.uint8(cv2.normalize(sqrt_image, None, 0, 255, cv2.NORM_MINMAX))
    
    return log_image, exp_image, square_image, sqrt_image


def Func_m(image):
    """Returns the mean μ and the standard deviation of an image."""
    mu = np.mean(image)
    std_dev = np.std(image)
    return mu, std_dev

### 1.4 Normalize, Inverse, CalcHist, and Threshold Functions

In [ ]:
def Normalize(image, new_min=0, new_max=255):
    """Normalizes image to the range [new_min, new_max]."""
    Inorm = cv2.normalize(image, None, new_min, new_max, cv2.NORM_MINMAX)
    Inorm = np.uint8(Inorm)
    return Inorm


def Inverse(image):
    """Returns Inv (the inverse/negative image): Inv = max(I) - I"""
    max_val = np.max(image)
    Inv = max_val - image
    return Inv


def CalcHist(image):
    """
    Returns histogram H and bins b.
    - H: vector representing the histogram
    - b: vector representing equally spaced intervals (bins)
    """
    H = cv2.calcHist([image], [0], None, [256], [0, 256])
    H = H.flatten()
    b = np.arange(256)
    return H, b


def Threshold(image, threshold_value):
    """Binary thresholding: values > threshold → 255, else → 0"""
    _, T = cv2.threshold(image, threshold_value, 255, cv2.THRESH_BINARY)
    return T

print("Normalize, Inverse, CalcHist, and Threshold defined successfully!")

### 1.5 Pipeline Functions

Complete processing pipelines that combine multiple operations.

In [ ]:
def Func_j(image_path):
    """
    Inversion Pipeline:
    1. Opens image 2. Displays it
    3. Shows histogram
    4. Inverts it
    5. Calculates inverted histogram
    6. Visualizes results
    """
    I, L, C = OpenImage(image_path)
    H_original, b = CalcHist(I)
    I_inv = Inverse(I)
    H_inverted, _ = CalcHist(I_inv)
    
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    plt.imshow(I, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(2, 2, 2)
    plt.plot(b, H_original)
    plt.title('Original Histogram')
    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    plt.grid(True)
    
    plt.subplot(2, 2, 3)
    plt.imshow(I_inv, cmap='gray')
    plt.title('Inverted Image')
    plt.axis('off')
    
    plt.subplot(2, 2, 4)
    plt.plot(b, H_inverted)
    plt.title('Inverted Histogram')
    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    return I, I_inv, H_original, H_inverted

In [ ]:
def Func_t(image_path):
    """
    Normalization Pipeline:
    1. Opens image
    2. Displays it
    3. Shows histogram
    4. Normalizes to [10, 50]
    5. Calculates normalized histogram
    6. Visualizes results
    """
    I, L, C = OpenImage(image_path)
    H_original, b = CalcHist(I)
    I_norm = Normalize(I, new_min=10, new_max=50)
    H_normalized, _ = CalcHist(I_norm)
    
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    plt.imshow(I, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(2, 2, 2)
    plt.plot(b, H_original)
    plt.title('Original Histogram')
    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    plt.grid(True)
    
    plt.subplot(2, 2, 3)
    plt.imshow(I_norm, cmap='gray')
    plt.title('Normalized Image [10, 50]')
    plt.axis('off')
    
    plt.subplot(2, 2, 4)
    plt.plot(b, H_normalized)
    plt.title('Normalized Histogram')
    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    plt.xlim([0, 255])
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    print("\nInterpretation:")
    print("Pixel values are compressed to [10, 50], reducing contrast.")
    
    return I, I_norm, H_original, H_normalized

In [ ]:
def Func_f(image_path):
    """
    Thresholding Pipeline:
    1. Opens image
    2. Displays it
    3. Shows histogram
    4. Applies thresholding (s=128)
    5. Calculates thresholded histogram
    6. Visualizes results
    """
    I, L, C = OpenImage(image_path)
    H_original, b = CalcHist(I)
    I_thresh = Threshold(I, threshold_value=128)
    H_thresholded, _ = CalcHist(I_thresh)
    
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    plt.imshow(I, cmap='gray')
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(2, 2, 2)
    plt.plot(b, H_original)
    plt.title('Original Histogram')
    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    plt.grid(True)
    
    plt.subplot(2, 2, 3)
    plt.imshow(I_thresh, cmap='gray')
    plt.title('Thresholded Image (s=128)')
    plt.axis('off')
    
    plt.subplot(2, 2, 4)
    plt.plot(b, H_thresholded)
    plt.title('Thresholded Histogram')
    plt.xlabel('Pixel Intensity')
    plt.ylabel('Frequency')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    return I, I_thresh, H_original, H_thresholded

## Exercise 2: Matrix Definition and Random Generation

### 2.1 Pattern Matrix

Define a matrix representing a pattern where:
- Black pixel = 0
- Gray pixel = 128
- White pixel = 255

In [ ]:
# Define pattern matrix
I_pattern = np.array([
    [0, 0, 0, 255, 255, 255, 128, 128, 128],
    [0, 0, 0, 255, 255, 255, 128, 128, 128],
    [0, 0, 0, 255, 255, 255, 128, 128, 128],
    [255, 255, 255, 128, 128, 128, 0, 0, 0],
    [255, 255, 255, 128, 128, 128, 0, 0, 0],
    [255, 255, 255, 128, 128, 128, 0, 0, 0],
    [128, 128, 128, 0, 0, 0, 255, 255, 255],
    [128, 128, 128, 0, 0, 0, 255, 255, 255],
    [128, 128, 128, 0, 0, 0, 255, 255, 255]
], dtype=np.uint8)

# Visualize pattern and its histogram
H_pattern, b_pattern = CalcHist(I_pattern)

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.imshow(I_pattern, cmap='gray')
plt.title('Pattern Image')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.bar(b_pattern, H_pattern, width=1.0)
plt.title('Histogram of Pattern Image')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.grid(True)
plt.tight_layout()
plt.show()

print(f"Pattern matrix shape: {I_pattern.shape}")

### 2.2 Random Matrix Generation (512×512)

Generate random matrices using different NumPy methods:
- **rand()**: Uniform distribution [0, 1] scaled to [0, 255]
- **randn()**: Normal distribution with mean=128 and random std
- **randint()**: Uniform integer distribution [0, 256)

In [ ]:
# Method 1: rand - uniform distribution [0, 1]
ran_d = np.random.rand(512, 512)
ran_d = np.uint8(ran_d * 255)

# Method 2: randn - normal distribution with mean=128 and random std
random_std = np.random.uniform(10, 50)
ran_dN = np.random.randn(512, 512) * random_std + 128
ran_dN = np.clip(ran_dN, 0, 255)
ran_dN = np.uint8(ran_dN)

# Method 3: randint - integers in range [0, 256)
ran_dINT = np.random.randint(low=0, high=256, size=(512, 512), dtype=np.uint8)

print(f"Generated randn with mean=128 and std={random_std:.2f}")
print(f"rand matrix shape: {ran_d.shape}")
print(f"randn matrix shape: {ran_dN.shape}")
print(f"randint matrix shape: {ran_dINT.shape}")

### 2.3 Visualize Random Matrices and Histograms

In [ ]:
# Calculate histograms
H_rand, b_rand = CalcHist(ran_d)
H_randn, b_randn = CalcHist(ran_dN)
H_randint, b_randint = CalcHist(ran_dINT)

# Visualize
plt.figure(figsize=(15, 10))

# rand
plt.subplot(3, 2, 1)
plt.imshow(ran_d, cmap='gray')
plt.title('rand(512, 512) - Uniform [0, 1] scaled to [0, 255]')
plt.axis('off')

plt.subplot(3, 2, 2)
plt.plot(b_rand, H_rand)
plt.title('Histogram - rand')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.grid(True)

# randn
plt.subplot(3, 2, 3)
plt.imshow(ran_dN, cmap='gray')
plt.title(f'randn(512, 512) - Normal(μ=128, σ={random_std:.2f})')
plt.axis('off')

plt.subplot(3, 2, 4)
plt.plot(b_randn, H_randn)
plt.title('Histogram - randn')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.grid(True)

# randint
plt.subplot(3, 2, 5)
plt.imshow(ran_dINT, cmap='gray')
plt.title('randint(0, 256, (512, 512)) - Uniform integers')
plt.axis('off')

plt.subplot(3, 2, 6)
plt.plot(b_randint, H_randint)
plt.title('Histogram - randint')
plt.xlabel('Pixel Intensity')
plt.ylabel('Frequency')
plt.grid(True)

plt.tight_layout()
plt.show()

### 2.4 Observations

**Analysis of the histograms:**

1. **rand() histogram:**
   - Nearly uniform distribution across all intensity values
   - All bins have approximately equal frequency
   - Creates pure random noise with no bias

2. **randn() histogram:**
   - **Bell-shaped (Gaussian) distribution** centered at mean (128)
   - Standard deviation controls the spread around the mean
   - Most pixels clustered around 128
   - Creates a gray, noisy appearance with natural-looking noise

3. **randint() histogram:**
   - Uniform distribution similar to rand()
   - Each intensity value has equal probability
   - Pure random noise pattern with integer values

**Key Observation**: The randn() histogram clearly shows a **normal distribution** with the characteristic bell curve shape, which is fundamentally different from the flat uniform distributions of rand() and randint().

## Test Pipeline Functions

Uncomment the lines below to test the pipeline functions with your own images:

In [ ]:
print("All functions defined and ready to use!")
print("\nTo test with your images:")
print(Func_j('../Lab1/flower.png'))
print(Func_t('../Lab1/flower.png'))
print(Func_f('../Lab1/flower.png'))